In [95]:
import pandas as pd
import numpy as np
from sqlalchemy import create_engine

In [103]:
engine = create_engine("sqlite:///Data/player_database.db")
input_data = pd.read_sql("BAYESIAN_3PT_INPUT_2023_24", engine)
bayesian_percentages = pd.read_sql("BAYESIAN_3PT_OUTPUT_2023_24", engine)
player_info = pd.read_sql("COMMON_PLAYER_INFO", engine)
free_agents = pd.read_sql("FREE_AGENTS_2023_24", engine)

In [104]:
free_agent_ids = free_agents["PLAYER_ID"].tolist()

In [105]:
free_agent_data = bayesian_percentages[bayesian_percentages["PLAYER_ID"].isin(free_agent_ids)]

In [106]:
#Players who are overvalued as 3PT shooters typically will have a lower Bayesian 3P% than season 3P%
overvalued_players = free_agent_data[free_agent_data["BAYESIAN_3P%"] < free_agent_data["SEASON_3P%"]]

#Players who are undervalued as 3PT shooters typically will have a higher Bayesian 3P% than season 3P%
undervalued_players = free_agent_data[free_agent_data["BAYESIAN_3P%"] > free_agent_data["SEASON_3P%"]]

In [107]:
overvalued_players.to_sql("OVERVALUED_FREE_AGENTS_2023_24", engine, if_exists="replace", index=False)
undervalued_players.to_sql("UNDERVALUED_FREE_AGENTS_2023_24", engine, if_exists="replace", index=False)
free_agent_data.to_sql("FREE_AGENT_3PT_DATA", engine, if_exists="replace", index=False)

178

In [108]:
overvalued_players.to_csv("Data/overvalued_free_agents_2023_24.csv")
undervalued_players.to_csv("Data/undervalued_free_agents_2023_24.csv")
free_agent_data.to_csv("Data/free_agent_3pt_data.csv")